# Forest management problem

In [1]:
%%html
<style>
.dataframe th {
    font-size: 24px;
}
.dataframe td {
    font-size: 22px;
}
</style>

In [2]:
import numpy as np
import pandas as pd
import shap
import matplotlib.pyplot as plt

from desdeo_problem.problem import DiscreteDataProblem
from desdeo_tools.scalarization import SimpleASF, DiscreteScalarizer, StomASF, PointMethodASF
from desdeo_tools.solver import DiscreteMinimizer
from shapley_values.explanations import *
from shapley_values.utilities import *
from IPython.core.display import display, HTML

# parameters
## the ASF used
ASF = PointMethodASF

# Load and setup problem (all objectives are defined to be maximized, so we multiply the whole dataframe by -1)
df = -1*pd.read_csv("../data/forest.csv")
pareto_front = df.to_numpy()

# compute the ideal and nadir
ideal = np.min(pareto_front[:, 0:3], axis=0)
nadir = np.max(pareto_front[:, 0:3], axis=0)

objective_names = ["Income", "Carbon", "Habitat index"]
n_objectives = 3

# we do not know the original decision variables, so we use a single 'dummy' variable just to make sure DiscreteDataProblem works correctly with the data
problem = DiscreteDataProblem(df, "dummy", objective_names, nadir, ideal)

asf = ASF(nadir, ideal)

# sample the Pareto front for missing data to be used while computing SHAP values
missing_data = shap.sample(pareto_front[:, 0:n_objectives], nsamples=200)

# generate the method to be used, or the black-box
bb = generate_black_box(problem, asf)

# define the explainer that computed SHAP values (use kernel SHAP)
explainer = shap.KernelExplainer(bb, missing_data)

data_multipliers = np.array([1e-7, 1e-9, 1e-4])
def scale_forest_data(data, multipliers=data_multipliers):
    return multipliers * data

# CHSI: combined suitable habitat index
objective_names_multi = ["Income (10^7)", "Stored CO2 (10^9)", "CSHI (10^4)"]

def plot(result, objective_names=objective_names_multi, ymin=0, ymax=1):
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    ax.set_ylim(ymin, ymax)
    ax.bar(objective_names, np.squeeze(result)) 
    plt.grid()
    plt.show()
    
def to_dataframe(points, indices):
    return pd.DataFrame({objective_names_multi[i]: [-scale_forest_data(points[j])[i] for j in range(len(points))] for i in range(3)}, index=indices).round(3)

Using 200 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


## Ideal and nadir

In [3]:
df_ideal_and_nadir = to_dataframe([ideal, nadir], ["Ideal", "Nadir"])
previous_solution = nadir

df_ideal_and_nadir

,Income (10^7),Stored CO2 (10^9),CSHI (10^4)
Ideal,6.285,8.269,3.244
Nadir,1.877,6.733,2.139


## Setting a reference point

In [4]:
reference_point = [3.5, 7.85, 3.0]
reference_point = -np.array(reference_point) * 1/data_multipliers

to_dataframe([ideal, reference_point, previous_solution], indices=["Ideal", "Reference point", "Nadir"]).round(3)

,Income (10^7),Stored CO2 (10^9),CSHI (10^4)
Ideal,6.285,8.269,3.244
Reference point,3.500,7.850,3.000
Nadir,1.877,6.733,2.139


## Computing the initial solution based on the reference point

In [5]:
new_solution = bb(np.atleast_2d(reference_point)).squeeze()
shap_values = np.array(explainer.shap_values(reference_point))

df = to_dataframe([ideal, reference_point, new_solution, nadir], indices=["Ideal", "Reference point", "New solution", "Nadir"])
display(HTML(df.to_html()))

,Income (10^7),Stored CO2 (10^9),CSHI (10^4)
Ideal,6.285,8.269,3.244
Reference point,3.500,7.850,3.000
New solution,3.720,7.983,3.057
Nadir,1.877,6.733,2.139


## Specification of the objective to be improved

In [6]:
to_improve = 2
output_str, _, _, _ = how_to_improve_objective_i(shap_values, to_improve, reference_point, new_solution, objective_names=objective_names)
suggestion, explanation = split_suggestion_explanation(output_str)

df = to_dataframe([ideal, reference_point, nadir], ["Ideal", "Reference point", "Nadir"])

display(HTML(f"<p style=font-size:20px>{suggestion}</p>"))
display(HTML(f"<p style=font-size:20px>{explanation}</p>"))

display(HTML(df.to_html()))


,Income (10^7),Stored CO2 (10^9),CSHI (10^4)
Ideal,6.285,8.269,3.244
Reference point,3.500,7.850,3.000
Nadir,1.877,6.733,2.139


## Specifying a new reference point and computing a new solution

In [7]:
previous_solution = new_solution

reference_point = [3.5, 7.75, 3.1]
reference_point = -np.array(reference_point) * 1/data_multipliers

new_solution = bb(np.atleast_2d(reference_point)).squeeze()
shap_values = np.array(explainer.shap_values(reference_point))

df = to_dataframe([ideal, previous_solution, new_solution, reference_point, nadir], indices=["Ideal", "Previous solution", "New solution", "Reference point", "Nadir"])
display(HTML(df.to_html()))

,Income (10^7),Stored CO2 (10^9),CSHI (10^4)
Ideal,6.285,8.269,3.244
Previous solution,3.720,7.983,3.057
New solution,3.579,7.810,3.136
Reference point,3.500,7.750,3.100
Nadir,1.877,6.733,2.139


In [8]:
to_improve = 1
output_str, _, _, _ = how_to_improve_objective_i(shap_values, to_improve, reference_point, new_solution, objective_names=objective_names)
suggestion, explanation = split_suggestion_explanation(output_str)

df = to_dataframe([ideal, reference_point, nadir], ["Ideal", "Reference point", "Nadir"])

display(HTML(f"<p style=font-size:20px>{suggestion}</p>"))
display(HTML(f"<p style=font-size:20px>{explanation}</p>"))

display(HTML(df.to_html()))

,Income (10^7),Stored CO2 (10^9),CSHI (10^4)
Ideal,6.285,8.269,3.244
Reference point,3.500,7.750,3.100
Nadir,1.877,6.733,2.139


In [9]:
previous_solution = new_solution

reference_point = [4.4, 7.705, 2.8]
reference_point = -np.array(reference_point) * 1/data_multipliers

new_solution = bb(np.atleast_2d(reference_point)).squeeze()
shap_values = np.array(explainer.shap_values(reference_point))

df = to_dataframe([ideal, previous_solution, new_solution, reference_point, nadir], indices=["Ideal", "Previous solution", "New solution", "Reference point", "Nadir"])
display(HTML(df.to_html()))

,Income (10^7),Stored CO2 (10^9),CSHI (10^4)
Ideal,6.285,8.269,3.244
Previous solution,3.579,7.810,3.136
New solution,4.613,7.772,2.854
Reference point,4.400,7.705,2.800
Nadir,1.877,6.733,2.139
